# 基于区域的卷积神经网络（R-CNNs）
:label:`sec_rcnn`

除了在 :numref:`sec_ssd` 中描述的单次多框检测外，基于区域的卷积神经网络或带有CNN特征的区域（R-CNNs）也是将深度学习应用于目标检测的许多开创性方法之一 :cite:`Girshick.Donahue.Darrell.ea.2014`。在本节中，我们将介绍R-CNN及其一系列改进：快速R-CNN :cite:`Girshick.2015`、更快的R-CNN :cite:`Ren.He.Girshick.ea.2015`和掩码R-CNN :cite:`He.Gkioxari.Dollar.ea.2017`。由于篇幅有限，我们将只关注这些模型的设计。

## R-CNNs

*R-CNN* 首先从输入图像中提取许多（例如2000个）*区域提议*（例如，锚框也可以被视为区域提议），并标记它们的类别和边界框（例如，偏移量）:cite:`Girshick.Donahue.Darrell.ea.2014`。然后使用一个CNN对每个区域提议进行前向传播以提取其特征。接下来，利用每个区域提议的特征来预测该区域提议的类别和边界框。

![R-CNN模型。](../img/r-cnn.svg)
:label:`fig_r-cnn`

:numref:`fig_r-cnn` 展示了R-CNN模型。具体来说，R-CNN由以下四个步骤组成：

1. 执行*选择性搜索*，在输入图像上提取多个高质量的区域提议 :cite:`Uijlings.Van-De-Sande.Gevers.ea.2013`。这些提议的区域通常是在不同尺度下具有不同形状和大小的。每个区域提议将被标记为一个类别和一个真实边界框。
1. 选择一个预训练的CNN，并在其输出层之前截断它。将每个区域提议调整到网络所需的输入尺寸，并通过前向传播输出该区域提议的提取特征。
1. 将每个区域提议的提取特征和标记的类别作为样本。训练多个支持向量机来分类对象，其中每个支持向量机单独确定样本是否包含特定类别。
1. 将每个区域提议的提取特征和标记的边界框作为样本。训练一个线性回归模型来预测真实边界框。

尽管R-CNN模型使用预训练的CNN有效地提取图像特征，但它很慢。想象一下，我们从单个输入图像中选择了数千个区域提议：这需要数千次的CNN前向传播来进行目标检测。这种巨大的计算负载使得在实际应用中广泛使用R-CNN变得不可行。

## 快速R-CNN

R-CNN的主要性能瓶颈在于对每个区域提议独立进行CNN前向传播，而不共享计算。由于这些区域通常有重叠，独立的特征提取导致了大量的重复计算。*快速R-CNN*相较于R-CNN的一个主要改进是仅在整个图像上执行一次CNN前向传播 :cite:`Girshick.2015`。

![快速R-CNN模型。](../img/fast-rcnn.svg)
:label:`fig_fast_r-cnn`

:numref:`fig_fast_r-cnn` 描述了快速R-CNN模型。其主要计算过程如下：

1. 与R-CNN相比，在快速R-CNN中用于特征提取的CNN的输入是整个图像，而不是单个区域提议。此外，这个CNN是可以训练的。给定一个输入图像，假设CNN的输出形状为$1 \times c \times h_1  \times w_1$。
1. 假设选择性搜索生成了$n$个区域提议。这些区域提议（具有不同的形状）在CNN输出中标记出兴趣区域（具有不同的形状）。然后这些兴趣区域进一步提取相同形状的特征（例如高度$h_2$和宽度$w_2$指定），以便容易地拼接。为了实现这一点，快速R-CNN引入了*兴趣区域池化（RoI）*层：CNN输出和区域提议被输入到这一层，输出拼接后的特征形状为$n \times c \times h_2 \times w_2$，这对所有区域提议进行了进一步提取。
1. 使用全连接层，将拼接后的特征转换成形状为$n \times d$的输出，其中$d$取决于模型设计。
1. 预测每个$n$个区域提议的类别和边界框。更具体地说，在类别和边界框预测中，将全连接层输出转换为形状为$n \times q$（$q$是类别的数量）和形状为$n \times 4$的输出，分别对应类别预测和边界框预测。类别预测使用softmax回归。

快速R-CNN提出的兴趣区域池化层不同于在 :numref:`sec_pooling` 中介绍的池化层。在池化层中，我们通过指定池化窗口的大小、填充和步幅间接控制输出形状。相比之下，在兴趣区域池化层中，我们可以直接指定输出形状。

例如，让我们指定每个区域的输出高度和宽度分别为$h_2$和$w_2$。对于任何形状为$h \times w$的兴趣区域窗口，此窗口被划分为一个$h_2 \times w_2$网格的小窗口，每个小窗口的形状大约为$(h/h_2) \times (w/w_2)$。实际上，任何小窗口的高度和宽度应向上取整，并且最大元素应作为小窗口的输出。因此，即使兴趣区域具有不同的形状，兴趣区域池化层也能提取相同形状的特征。

作为一个说明性的例子，在 :numref:`fig_roi` 中，选择了一个$4 \times 4$输入上的左上角$3\times 3$兴趣区域。对于这个兴趣区域，我们使用$2\times 2$的兴趣区域池化层获得一个$2\times 2$的输出。注意，四个分割的小窗口分别包含元素0, 1, 4, 和 5（5是最大值）；2 和 6（6 是最大值）；8 和 9（9 是最大值）；以及 10。

![一个$2\times 2$的兴趣区域池化层。](../img/roi.svg)
:label:`fig_roi`

下面演示兴趣区域池化层的计算。假设CNN提取的特征`X`的高度和宽度均为4，并且只有一个通道。

In [1]:
import torch
import torchvision

X = torch.arange(16.).reshape(1, 1, 4, 4)
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]]]])

让我们进一步假设输入图像的高度和宽度都是40像素，并且选择性搜索在此图像上生成了两个区域提议。每个区域提议都用五个元素表示：其对象类别，后面跟着其左上角和右下角的$(x, y)$坐标。

In [2]:
rois = torch.Tensor([[0, 0, 0, 20, 20], [0, 0, 10, 30, 30]])

由于`X`的高度和宽度是输入图像高度和宽度的$1/10$，根据指定的`spatial_scale`参数，两个区域提议的坐标被乘以0.1。然后，在`X`上将这两个感兴趣区域分别标记为`X[:, :, 0:3, 0:3]`和`X[:, :, 1:4, 0:4]`。最后，在$2\times 2$的感兴趣区域池化中，每个感兴趣区域被划分为子窗口网格，以进一步提取形状为$2\times 2$的特征。

In [3]:
torchvision.ops.roi_pool(X, rois, output_size=(2, 2), spatial_scale=0.1)

tensor([[[[ 5.,  6.],
          [ 9., 10.]]],


        [[[ 9., 11.],
          [13., 15.]]]])

## 更快的 R-CNN

为了提高目标检测的准确性，Fast R-CNN 模型通常需要生成大量的选择性搜索区域建议。为了在不损失准确性的情况下减少区域建议，*更快的 R-CNN* 提议用*区域建议网络*取代选择性搜索 :cite:`Ren.He.Girshick.ea.2015`。

![更快的 R-CNN 模型。](../img/faster-rcnn.svg)
:label:`fig_faster_r-cnn`

:numref:`fig_faster_r-cnn` 显示了更快的 R-CNN 模型。与 Fast R-CNN 相比，更快的 R-CNN 仅将区域建议方法从选择性搜索更改为区域建议网络。模型的其余部分保持不变。区域建议网络的工作步骤如下：

1. 使用一个填充为1的$3\times 3$卷积层将CNN输出转换为具有$c$个通道的新输出。这样，CNN提取特征图的空间维度上的每个单元都获得了一个长度为$c$的新特征向量。
1. 以特征图上每个像素为中心，生成多个不同尺度和宽高比的锚框并进行标记。
1. 利用每个锚框中心处的长度为$c$的特征向量，预测该锚框的二元类别（背景或物体）及边界框。
1. 考虑那些被预测为物体的边界框。使用非极大值抑制去除重叠结果。剩余的物体预测边界框即为兴趣区域池化层所需的区域建议。

值得注意的是，作为更快的 R-CNN 模型的一部分，区域建议网络与模型的其余部分一起联合训练。换句话说，更快的 R-CNN 的目标函数不仅包括目标检测中的类别和边界框预测，还包括区域建议网络中锚框的二元类别和边界框预测。由于端到端训练的结果，区域建议网络学会了如何生成高质量的区域建议，从而能够在减少由数据学习得到的区域建议数量的同时保持目标检测的准确性。

## Mask R-CNN

如果训练数据集中还标注了图像上物体的像素级位置，*Mask R-CNN* 可以有效利用这些详细标签来进一步提高目标检测的准确性 :cite:`He.Gkioxari.Dollar.ea.2017`。

![Mask R-CNN 模型。](../img/mask-rcnn.svg)
:label:`fig_mask_r-cnn`

如 :numref:`fig_mask_r-cnn` 所示，Mask R-CNN 是基于更快的 R-CNN 进行修改的。具体来说，Mask R-CNN 将兴趣区域池化层替换为*兴趣区域（RoI）对齐*层。此兴趣区域对齐层使用双线性插值来保留特征图上的空间信息，更适合于像素级预测。该层的输出包含所有兴趣区域相同形状的特征图。它们不仅用于预测每个兴趣区域的类别和边界框，还通过额外的全卷积网络预测物体的像素级位置。关于使用全卷积网络预测图像像素级语义的更多细节将在本章后续部分提供。

## 总结

* R-CNN 从输入图像中提取许多区域建议，使用 CNN 对每个区域建议执行前向传播以提取其特征，然后使用这些特征预测该区域建议的类别和边界框。
* Fast R-CNN 对 R-CNN 的主要改进之一是仅对整个图像执行一次 CNN 前向传播。它还引入了兴趣区域池化层，以便可以进一步提取具有不同形状的兴趣区域的相同形状特征。
* 更快的 R-CNN 用联合训练的区域建议网络替换了 Fast R-CNN 中的选择性搜索，使得前者能够在减少区域建议数量的同时保持目标检测的准确性。
* 基于更快的 R-CNN，Mask R-CNN 额外引入了全卷积网络，以利用像素级标签进一步提高目标检测的准确性。

## 练习

1. 我们能否将目标检测构架为单一回归问题，例如预测边界框和类别概率？你可以参考 YOLO 模型的设计 :cite:`Redmon.Divvala.Girshick.ea.2016`。
1. 比较单次多框检测与本节介绍的方法。它们的主要区别是什么？你可以参考 :citet:`Zhao.Zheng.Xu.ea.2019` 中的图2。

[讨论](https://discuss.d2l.ai/t/1409)